***FUNCION DE LIMPIEZA***

In [339]:
import requests
import pandas as pd
import time
import random
from github import Github, GithubException
from datetime import datetime
import psycopg2
import os

In [340]:
#leer el csv en la carpeta actual
#Agregar un archivo que está en github
#df_busqueda = pd.read_csv('https://github.com/labdatos-se/economia/blob/main/mapa_gro/guerrero.csv')
df_busqueda = pd.read_csv('D:\Proy\GitHub\labdatos-se\economia\mapa_gro\guerrero.csv')
#df_busqueda = pd.read_csv('guerrero.csv')
df_busqueda.head()

,id_tienda,sucursal,store_name,marca,producto,precio,contenido,busqueda,latitud,longitud,address,fecha_consulta
0,9993,Paradox Santa Fe,Soriana,1.2.3.,aceite vegetal comestible,46.0,botella 1 l,aceite,19.357303,-99.273539,"av sta fe 546, lomas de santa fe, contadero, c...",31/10/2023 18:41
1,9993,Paradox Santa Fe,Soriana,capullo,aceite de canola,60.5,botella 840 ml,aceite,19.357303,-99.273539,"av sta fe 546, lomas de santa fe, contadero, c...",31/10/2023 18:41
2,9993,Paradox Santa Fe,Soriana,oléico,aceite puro de cártamo,88.9,botella 825 ml,aceite,19.357303,-99.273539,"av sta fe 546, lomas de santa fe, contadero, c...",31/10/2023 18:41
3,9993,Paradox Santa Fe,Soriana,pam,aceite original comestible,73.9,aerosol 170 g,aceite,19.357303,-99.273539,"av sta fe 546, lomas de santa fe, contadero, c...",31/10/2023 18:41
4,9993,Paradox Santa Fe,Soriana,1.2.3.,aceite vegetal comestible,46.0,botella 1 l,aceite,19.357303,-99.273539,"av sta fe 546, lomas de santa fe, contadero, c...",31/10/2023 18:41


In [341]:
reemplazos = {'Ã¡':'á', 'Ã©':'é', 'Ã-­':'í', 'Ã³':'ó', 'Ãº':'ú', 'Ã±':'ñ'}
df_busqueda = df_busqueda.replace(reemplazos, regex=True)

In [342]:
#Si hay un NaN, eliminar toda la fila
df_busqueda = df_busqueda.dropna()

In [345]:
#Función de limpieza de prueba
def limpiaprod(df):
    #Aceite
    aceite = df[df['producto'].str.contains("aceite") & (df['contenido'].str.contains("946"))]
    aceite = aceite[~aceite["producto"].str.contains("oliva")]

    #Arroz
    arroz = df[df['producto'].str.contains("arroz") & (df['contenido'].str.contains("1 kg"))]
    arroz = arroz[~arroz["producto"].str.contains("sushi") & ~arroz["producto"].str.contains("arborio") & ~arroz["producto"].str.contains("precocido")& ~arroz["producto"].str.contains("orgánico")]

    #Atún
    atún = df[df['producto'].str.contains("atún") & (df['contenido'].str.contains("140")) & (df['contenido'].str.contains("lata")) & (df['producto'].str.contains("agua"))]
    atún = atún[~atún["producto"].str.contains("aceite")]

    #Azúcar
    azúcar = df[df['producto'].str.contains("azúcar") & (df['contenido'].str.contains("2 kg"))]

    #Cepillo tipo plancha
    cepillo_plancha = df[df['producto'].str.contains("cepillo tipo plancha")]
    cepillo_plancha = cepillo_plancha[~cepillo_plancha["contenido"].str.contains("cja")]

    #cepillo dental
    cepillo_dental = df[df['producto'].str.contains("cepillo dental") & (df['busqueda'].str.contains("cepillo"))]
    cepillo_dental = cepillo_dental[~cepillo_dental["producto"].str.contains("infantil") & ~cepillo_dental["producto"].str.contains("eléctrico") & ~cepillo_dental["producto"].str.contains("kid") & ~cepillo_dental["producto"].str.contains("pasta") & ~cepillo_dental["producto"].str.contains("electrónico") & ~cepillo_dental["producto"].str.contains("pow")]

    #chocolate en polvo
    #REVISAR
    chocolate = df[(df['producto'].str.contains("chocolate en polvo") | df['producto'].str.contains("polvo sabor chocolate")) & (df['contenido'].str.contains("700 g"))]
    chocolate = chocolate[~chocolate["producto"].str.contains("cereal")]   
    
    #cloro
    cloro = df[df['producto'].str.contains("cloro") & (df['busqueda'].str.contains("cloro"))]
    cloro = cloro[~cloro["producto"].str.contains("toallitas") & ~cloro["producto"].str.contains("tricloro") & ~cloro["producto"].str.contains("sin") & ~cloro["producto"].str.contains("detergente") & ~cloro["producto"].str.contains("polvo") & ~cloro["producto"].str.contains("alternativa") & ~cloro["producto"].str.contains("anti") & ~cloro["producto"].str.contains("con") & ~cloro["producto"].str.contains("perla") & ~cloro["producto"].str.contains("suplemento") & ~cloro["producto"].str.contains("tableta") & ~cloro["producto"].str.contains("piramina") & ~cloro["producto"].str.contains("crema")]      
    
    #desodorante
    desodorante = df[(df['producto'].str.contains("desodorante") | df['producto'].str.contains("antitranspirante")) & df['contenido'].str.match("barra 50 g")]

    #escobas
    escobas = df[df['producto'].str.contains("escoba") & (df['contenido'].str.match("unidad") | df['contenido'].str.match("pieza") | df['contenido'].str.match("1 pieza")) & (df['busqueda'].str.contains("escobas"))]
    escobas = escobas[~escobas["busqueda"].str.contains("fibra")]

    #fibra
    fibra = df[df['producto'].str.contains("fibra") & df['busqueda'].str.match("fibra") & (df['contenido'].str.match("unidad") | df['contenido'].str.match("1 pieza"))]
    fibra = fibra[~fibra["producto"].str.contains("paño")]

    #franelas
    franelas = df[df['producto'].str.contains("franela")]

    #café
    café = df[df['busqueda'].str.contains("café") & (df['contenido'].str.contains("frasco 225 g") | df['contenido'].str.contains("frasco 200 g") | df['contenido'].str.contains("frasco 190g ") | df['contenido'].str.contains("frasco 180 g") | df['contenido'].str.contains("frasco 175 g"))]
    café = café[~café["producto"].str.contains("con") & ~café["producto"].str.contains("pack")]

    #frijol
    frijol = df[(df['busqueda'].str.contains("frijol")) & (df['producto'].str.contains("frijol")) & (df['contenido'].str.contains("1 kg") | df['contenido'].str.contains("900 g"))]
    frijol = frijol[~frijol["producto"].str.contains("refritos")]

    #guantes de plástico
    guantes = df[df['producto'].str.contains("guantes") & (df['busqueda'].str.contains("guantes"))]

    #harina para atole
    harina = df[df['producto'].str.contains("harina de arroz") & df['busqueda'].str.match("harina") & (df['contenido'].str.contains("200 g"))]

    #jabón de barra
    jabón_barra = df[(df['producto'].str.contains("jabón")) & (df['contenido'].str.contains("150 g") & (df['contenido'].str.contains("barra")))]
    jabón_barra = jabón_barra[~jabón_barra["producto"].str.contains("derm") & ~jabón_barra["producto"].str.contains("facial")]

    #jabón en polvo
    jabón_polvo = df[(df['producto'].str.contains("detergente")) & (df['contenido'].str.contains("1 kg") | df['contenido'].str.contains("2 kg") | df['contenido'].str.contains("2.1 kg") | df['contenido'].str.contains("750 g") | df['contenido'].str.contains("800 g"))]

    #jalador
    jalador = df[df['producto'].str.contains("jalador") & (df['busqueda'].str.contains("jalador")) & (df['contenido'].str.contains("1"))]
    jalador = jalador[~jalador["producto"].str.contains("vidrio") & ~jalador["producto"].str.contains("inye") & ~jalador["producto"].str.contains("paño")]

    #jergas
    jergas = df[df['producto'].str.contains("jerga") & (df['busqueda'].str.contains("jerga")) & (df['contenido'].str.contains("1"))]

    #lata de chiles
    lata_chiles = df[df['producto'].str.contains("chiles") & (df['busqueda'].str.contains("chiles")) & (df['busqueda'].str.contains("lata")) & (df['contenido'].str.contains("220 g"))]
    lata_chiles = lata_chiles[~lata_chiles["producto"].str.contains("adob")]

    #lenteja
    lenteja = df[df['producto'].str.contains("lenteja")]# & (df['contenido'].str.contains("900 g"))]
    lenteja = lenteja[~lenteja["producto"].str.contains("sopa") & ~lenteja["producto"].str.contains("ensalada") & ~lenteja["producto"].str.contains("chorizo") & ~lenteja["producto"].str.contains("orgánica") & ~lenteja["producto"].str.contains("hamburguesa")]

    #limpiador para pisos
    limpiador_pisos = df[df['producto'].str.contains("limpiador") & (df['busqueda'].str.contains("limpiador"))]
    limpiador_pisos = limpiador_pisos[~limpiador_pisos['busqueda'].str.contains("pieza") & ~limpiador_pisos['busqueda'].str.contains("aerosol") & ~limpiador_pisos['busqueda'].str.contains("atomizador") & ~limpiador_pisos['busqueda'].str.contains("doypack") & ~limpiador_pisos['busqueda'].str.contains("poett") & ~limpiador_pisos['busqueda'].str.contains("pouch") & ~limpiador_pisos['busqueda'].str.contains("spray")]

    #mayonesa
    mayonesa = df[(df['busqueda'].str.contains("mayonesa")) & (df['producto'].str.contains("mayonesa")) & (df['contenido'].str.contains("frasco")) & (df['contenido'].str.contains("650") | df['contenido'].str.contains("725") | df['contenido'].str.contains("730") | df['contenido'].str.contains("750") | df['contenido'].str.contains("775") | df['contenido'].str.contains("790") | df['contenido'].str.contains("800"))]

    #mermelada
    mermelada = df[df['producto'].str.contains("mermelada") & (df['busqueda'].str.contains("mermelada")) & (df['contenido'].str.contains("440"))]

    #pasta dental
    pasta_dental = df[(df['busqueda'].str.contains("pasta")) & (df['producto'].str.contains("pasta")) & (df['contenido'].str.contains("125 ml"))]
    pasta_dental = pasta_dental[~pasta_dental["producto"].str.contains("cepillo")]

    #peine
    peine = df[df['producto'].str.contains("peine") & (df['busqueda'].str.contains("peine")) & (df['contenido'].str.contains("1"))]
    peine = peine[~peine["producto"].str.contains("drer") & ~peine["producto"].str.contains("mickey") & ~peine["producto"].str.contains("winnie") & ~peine["producto"].str.contains("térmico")]

    #rastrillos
    #rastrillos = df[df['producto'].str.contains("rastrillo") & (df['busqueda'].str.contains("rastrillo")) & (df['contenido'].str.contains("3 piezas"))]
    rastrillos = df[df['producto'].str.contains("rastrillo") & (df['busqueda'].str.contains("rastillo")) & (df['contenido'].str.contains("3 piezas"))]  #RASTILLO
    rastrillos = rastrillos[~rastrillos["producto"].str.contains("máquina") & ~rastrillos["producto"].str.contains("cartucho")]

    #recogedor
    recogedor = df[df['producto'].str.contains("recogedor") & (df['busqueda'].str.contains("recogedor")) & (df['contenido'].str.contains("1"))]

    #rollos de papel
    rollos_papel = df[df['producto'].str.contains("papel") & (df['busqueda'].str.contains("rollos")) & (df['contenido'].str.contains("4"))]
    rollos_papel = rollos_papel[~rollos_papel["contenido"].str.contains("24") & ~rollos_papel["contenido"].str.contains("42")]

    #sacate
    sacate = df[df['busqueda'].str.contains("sacate") & ((df['producto'].str.contains("esponja")) | (df['producto'].str.contains("toalla"))) & (df['contenido'].str.contains("1"))]

    #sal
    sal = df[df['busqueda'].str.contains("sal") & (df['producto'].str.contains("sal"))]
    sal = sal[~sal["producto"].str.contains("mantequilla") & ~sal["producto"].str.contains("margarina") & ~sal["producto"].str.contains("parrillera") & ~sal["producto"].str.contains("himalaya") & ~sal["producto"].str.contains("con") & ~sal["producto"].str.contains("sin") & ~sal["producto"].str.contains("botana") & ~sal["producto"].str.contains("fritura") & ~sal["producto"].str.contains("papas") & ~sal["producto"].str.contains("palomitas") & ~sal["producto"].str.contains("cacah") & ~sal["producto"].str.contains("jabón") & ~sal["producto"].str.contains("flor") & ~sal["producto"].str.contains("agua") & ~sal["producto"].str.contains("hornea") & ~sal["producto"].str.contains("cristal")  & ~sal["producto"].str.contains("uva") & ~sal["producto"].str.contains("sustituto") & ~sal["producto"].str.contains("gou") & ~sal["producto"].str.contains("ahumada") & ~sal["producto"].str.contains("alpo") & ~sal["producto"].str.contains("light")]

    #sardina
    sardina = df[df['busqueda'].str.contains("sardina") & (df['producto'].str.contains("sardina")) & (df['contenido'].str.contains("425"))]

    #sopas instantáneas
    sopas = df[df['busqueda'].str.contains("sopa") & (df['producto'].str.contains("insta"))]

    #toallas faciales
    toallas_fac = df[df['busqueda'].str.contains("toalla") & (df['producto'].str.contains("facial"))]

    #toallas femeninas
    toallas_fem = df[df['busqueda'].str.contains("toalla") & (df['producto'].str.contains("fem")) & (df['contenido'].str.contains("8"))]
    toallas_fem = toallas_fem[~toallas_fem["contenido"].str.contains("28") & ~toallas_fem["contenido"].str.contains("18")]

    #toallas húmedas
    toallas_hum = df[df['busqueda'].str.contains("toalla") & (df['producto'].str.contains("eda")) & (df['contenido'].str.contains("80"))]
    toallas_hum = toallas_hum[~toallas_hum["contenido"].str.contains("3")]

    #Concatenar
    finaldf = pd.concat([aceite, arroz, atún, azúcar, cepillo_plancha, cepillo_dental, chocolate, cloro, desodorante, escobas, fibra, franelas, café, frijol, guantes, harina, jabón_barra, jabón_polvo, jalador, jergas, lata_chiles, lenteja, limpiador_pisos, mayonesa, mermelada, pasta_dental, peine, rastrillos, recogedor, rollos_papel, sacate, sal, sardina, sopas, toallas_fac, toallas_fem, toallas_hum])
    #finaldf = pd.concat([chocolate])
    #reset index
    finaldf.reset_index(drop=True, inplace=True)
    return finaldf


In [346]:
limpiaprod(df_busqueda)

,id_tienda,sucursal,store_name,marca,producto,precio,contenido,busqueda,latitud,longitud,address,fecha_consulta
0,4449,La Comer Tlalpan,La Comer,canoil,aceite vegetal puro de canola,47.0,botella 946 ml,aceite,19.288687,-99.179181,"plazuela de la fama 1, la fama, 14410 tlalpan,...",31/10/2023 18:41
1,9991,La Villa,Soriana,canoil,aceite vegetal puro de canola,47.9,botella 946 ml,aceite,19.465417,-99.125438,"calz de guadalupe 192, vallejo, gustavo a. mad...",31/10/2023 18:41
2,9993,Paradox Santa Fe,Soriana,verde valle,arroz grano grueso,40.7,bolsa 1 kg,arroz,19.357303,-99.273539,"av sta fe 546, lomas de santa fe, contadero, c...",31/10/2023 18:41
3,4449,La Comer Tlalpan,La Comer,verde valle,arroz grano grueso,44.0,bolsa 1 kg,arroz,19.288687,-99.179181,"plazuela de la fama 1, la fama, 14410 tlalpan,...",31/10/2023 18:41
4,4449,La Comer Tlalpan,La Comer,schettino,arroz tipo morelos,32.0,1 kg,arroz,19.288687,-99.179181,"plazuela de la fama 1, la fama, 14410 tlalpan,...",31/10/2023 18:41
...,...,...,...,...,...,...,...,...,...,...,...,...
921,9991,La Villa,Soriana,tencel,toallitas húmedas dody´s avena sábila,49.9,80 piezas,toallas+húmedas,19.465417,-99.125438,"calz de guadalupe 192, vallejo, gustavo a. mad...",31/10/2023 18:41
922,9991,La Villa,Soriana,palmolive,toallitas húmedas neutro balance,72.5,paquete 80 piezas,toallas+húmedas,19.465417,-99.125438,"calz de guadalupe 192, vallejo, gustavo a. mad...",31/10/2023 18:41
923,9991,La Villa,Soriana,dody's,toallas húmedas dermo active rosa,32.9,paquete 80 piezas,toallas+húmedas,19.465417,-99.125438,"calz de guadalupe 192, vallejo, gustavo a. mad...",31/10/2023 18:41
924,9991,La Villa,Soriana,soft dreams,toallas húmedas,35.0,80 piezas,toallas+húmedas,19.465417,-99.125438,"calz de guadalupe 192, vallejo, gustavo a. mad...",31/10/2023 18:41


In [349]:
#Guardar el dataframe en un csv
limpiaprod(df_busqueda).to_csv('D:\Proy\GitHub\labdatos-se\economia\mapa_gro\corner_limpio.csv', encoding='latin1', index=False)